<a href="https://colab.research.google.com/github/1234567890-sys/colab/blob/main/MinutesTranscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install ipywidgets

In [3]:
import ipywidgets as widgets

In [4]:
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output=widgets.Output()

def start_recording(data):
  messages.put(True)
  with output:
    display("Starting...")
    record = Thread(target=record_microphone)
    record.start()

    transcribe = Thread(target=speech_recognition, args=(output,))
    transcribe.start()

def stop_recording(data):
  with output:
    messages.get()
    display("Stopped.")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [5]:
!sudo apt-get install python3.11-dev
!sudo apt install portaudio19-dev
!python3 -m  pip install PyAudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.11-dev is already the newest version (3.11.10-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [6]:
!pip install pyaudio

In [7]:
import pyaudio

p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

In [8]:
CHANNEL = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
  p = pyaudio.PyAudio()

  stream = p.open(format=AUDIO_FORMAT,
                  channels=CHANNEL,
                  rate=FRAME_RATE,
                  input=True,
                  input_device_index=2,
                  frames_per_buffer=chunk)
  frames = []

  while not messages.empty():
    data = stream.read(chunk)
    frames.append(data)
    if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
      recordings.put(frames.copy())
      frames = []

  stream.stop_stream()
  stream.close()
  p.terminate()
#

In [9]:
!pip install vosk
!pip install transformers
!pip install torch

In [10]:
import subprocess
import json
from vosk import Model, KaldiRecognizer

model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
  while not messages.empty():
    frames = recordings.get()

    rec.AcceptWaveform(b''.join(frames))
    result = rec.Result()
    text = json.loads(result)["text"]

    cased = subprocess.check_output('python3 asr.py ' + text, shell=True, text=True, input=text)
    output.append_stdout(cased)